In [167]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.decomposition import PCA

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from matlab.matlab_yaro_lib import read_mat
from corr_lib import sprMat
from qt_wrapper import gui_fpath

Appended library directory /media/aleksejs/DataHDD/work/codes/theoretical-models/experimental-bridge/nest-mesoscopic-metric-test/codes/lib


## 0. Loading data

In [3]:
# Read LVM file from command line
data_path = gui_fpath("Select data directory", "./")
data, behaviour = read_mat(data_path)

# Get parameters
nTrials, nTimes, nChannels = data.shape
print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

Reading Yaro data from /mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15/mtp_15_2018_05_16_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (338, 201, 12)


# 1. Autoregression

What to include into model:
1. Autoregressive Dynamics
$$\vec{x}(t+1) = A\vec{x}(t)$$
2. Neural noise, to model input from external sources
$$\vec{x}(t+1) = A\vec{x}(t) + \nu, \;\;\;\;\; \nu \sim \mathcal{N}(\vec{\mu}, \Sigma)$$
3. Input and input-coupling
$$ \vec{x}(t+1) = A\vec{x}(t) + B\vec{u}(t) + C \vec{x}(t) \vec{u}(t) + \nu$$
4. Observable convolution
$$
\begin{eqnarray}
  \tau \dot{y} &=& -y + x \\
  \vec{y}(t+1) &\approx& \alpha \vec{x}(t) + (1 - \alpha) \vec{y}(t), \;\;\;\;\; \alpha = \frac{\Delta t}{\tau}
\end{eqnarray}$$

**TODO**:
* See what tools for autoregressors exist
* Read literature on DCM, understand how it works, how it is different from autoregressor
* For observable autoregressor, is it not equivalent to simply deconvolving data and then using basic deconvolved?
   $$x(t) = y(t) + \frac{y(t+1) - y(t)}{\alpha}$$
   * Ask Yaro if he thinks this makes sense
   * Look for most optimal deconvolution tool for this process